# lexicons

> Extract features from texts based on lexicons.

In [ ]:
#| default_exp lexicons

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from sklearn.base import BaseEstimator, TransformerMixin
from textplumber.store import TextFeatureStore
import numpy as np

In [ ]:
#| export
class LexiconCountVectorizer(BaseEstimator, TransformerMixin):
	""" A Sci-kit Learn pipeline component to get document-level counts for one or more lexicons. 
		This component should be used after the SpacyPreprocessor component with the same feature store. """ 
	def __init__(self,
			  	 feature_store: TextFeatureStore, # the feature store to use - this should be the same feature store used in the SpacyPreprocessor component
				 lexicons:dict, # the lexicons to use - a dictionary with the lexicon name as the key and the lexicon (a list of tokens to count) as the value
				 lowercase:bool = True, # whether to lowercase the tokens
				 ):

		self.feature_store = feature_store

		if not isinstance(lexicons, dict):
			raise ValueError("Lexicons should be a dictionary with lexicon name as the key and the lexicon (a list of tokens to count) as the value.")

		self.lexicons = lexicons
		self.lowercase = lowercase
	
	def fit(self, X, y=None):
		""" Fit the vectorizer to the tokens in the feature store. """
		return self
	
	def transform(self, X):
		""" Transform the texts to a matrix of counts. """
		docs_tokens = self.feature_store.get_tokens_from_texts(X, lowercase = self.lowercase)
		X = []
		for doc in docs_tokens:
			lexicon_counts = []
			for lexicon in self.lexicons:
				lexicon_counts.append(sum([1 for token in doc if token in self.lexicons[lexicon]]))
			X.append(lexicon_counts)
		return np.array(X)
		
	def get_feature_names_out(self, input_features=None):
		""" Get the feature names out from the vectorizer. """
		return list(self.lexicons.keys())

TODO: add an example.

In [ ]:
#| hide
from textplumber.preprocess import SpacyPreprocessor
from sklearn.pipeline import Pipeline

feature_store = TextFeatureStore()
spacy_preprocessor = SpacyPreprocessor(feature_store=feature_store)
lexicon = {
    'positive': ['good', 'happy', 'joyful'],
    'negative': ['bad', 'sad', 'angry']
}
lexicon_count_vectorizer = LexiconCountVectorizer(feature_store=feature_store, lexicons=lexicon)
pipeline = Pipeline([
    ('spacy_preprocessor', spacy_preprocessor),
    ('lexicon_count_vectorizer', lexicon_count_vectorizer)
])
docs = ['I am happy', 'I am sad', 'I am angry', 'I am happy happy']
pipeline.fit(docs)
X = pipeline.transform(docs)

id = lexicon_count_vectorizer.get_feature_names_out().index('positive')
assert X.tolist()[0][id] == 1
assert X.tolist()[0][id] == 1
assert X.tolist()[3][id] == 2
id = lexicon_count_vectorizer.get_feature_names_out().index('negative')
assert X.tolist()[0][id] == 0

/home/geoff/miniconda3/envs/textplumber/lib/python3.11/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()